В рамках выполнения данного задания вам необходимо обучить нейросетевую модель распознавания лиц

В качестве результата свой работы вам нужно предоставить файл classifier.py с классом Classifier внутри
В методе `__init__` данного класса должна происходить инициализация сети
Также у класса должен быть метод `calculate_descriptors`, который должен принимать на вход батч нормализованных изображений, а на выходе возвращать тензор с дескрипторами для всех изображений в батче

15 баллов вы получите за результат, который на тестовой выборке будет превосходить результаты baseline'а
3 лучших решения так же получат дополнительные баллы

In [ ]:
%pip install mxnet timm

In [ ]:
import torch
import sklearn
import sklearn.metrics
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'bilinear'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

# Dataloader CASIA-WebFace

In [ ]:
!git clone https://github.com/deepinsight/insightface.git

In [ ]:
!gdown 1LQHRJG7gUNWe4gIRK1jfrhImR7SOjOuF

In [ ]:
!unzip faces_webface_112x112.zip

In [ ]:
import sys ; sys.path.append('insightface/recognition/arcface_torch')
from insightface.recognition.arcface_torch.dataset import get_dataloader

In [ ]:
train_loader = get_dataloader(
    root_dir='faces_webface_112x112',
    local_rank=0,
    batch_size=256,
    num_workers=32
)

In [ ]:
for imgs, labels in train_loader:
    break

In [ ]:
plt.imshow(((imgs[0] + 1) / 2).detach().cpu().numpy().transpose(1, 2, 0))
plt.show()

Код нормализации изображений: https://github.com/davidsandberg/facenet/tree/master/src/align

# Model, loss, optimizer preparation

In [ ]:
import timm
model = timm.create_model(
    'resnet50',
    pretrained=True,
    num_classes=10572
).cuda()

In [ ]:
loss_CE = torch.nn.CrossEntropyLoss()

In [ ]:
opt_parameters = model.parameters()
optimizer = torch.optim.AdamW(
    opt_parameters,
    lr=1e-3,
    betas=(0.9, 0.999),
    weight_decay=0.0005
)
lr_scale_change = 0.5

# Train loop

In [ ]:
EPOCHS = 10

step, n_steps = 0, EPOCHS * len(train_loader)
for epoch in range(EPOCHS):
    for imgs, labels in train_loader:
        optimizer.zero_grad()

        output = model(imgs)

        loss = loss_CE(output, labels)
        loss.backward()

        optimizer.step()

        step += 1
        if step % 100 == 0:
            print(f'Step {step}/{n_steps}, loss {loss}')
    for param_group in optimizer.param_groups:
        param_group['lr'] *= lr_scale_change

# Save model

In [ ]:
class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [ ]:
model.fc = Identity()

In [ ]:
torch.save(model, 'model.pth')

# Test model

In [ ]:
class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [ ]:
class Classifier:
    def __init__(self, model_path):
        self.model = torch.load(model_path)
        self.model.eval()

    def calculate_descriptors(self, imgs):
        return self.model(imgs)

In [ ]:
classifier = Classifier('model.pth')
descriptors = classifier.calculate_descriptors(imgs)
descriptors = descriptors.detach().cpu().numpy()
descriptors = sklearn.preprocessing.normalize(descriptors)
print(descriptors.shape)

In [ ]:
distances = sklearn.metrics.pairwise_distances(descriptors)
print(distances.shape)

In [ ]:
distances[np.tril_indices(distances.shape[0])] = np.inf

In [ ]:
min_idxs = np.unravel_index(distances.argmin(), distances.shape)

In [ ]:
distances[min_idxs]

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(((imgs[min_idxs[0]] + 1) / 2).detach().cpu().numpy().transpose(1, 2, 0))
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(((imgs[min_idxs[1]] + 1) / 2).detach().cpu().numpy().transpose(1, 2, 0))
plt.axis('off')
plt.show()

In [ ]:
distances[np.tril_indices(distances.shape[0])] = -np.inf

In [ ]:
max_idxs = np.unravel_index(distances.argmax(), distances.shape)

In [ ]:
distances[max_idxs]

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(((imgs[max_idxs[0]] + 1) / 2).detach().cpu().numpy().transpose(1, 2, 0))
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(((imgs[max_idxs[1]] + 1) / 2).detach().cpu().numpy().transpose(1, 2, 0))
plt.axis('off')
plt.show()

Функция тестирования: https://github.com/deepinsight/insightface/blob/32682e54e08f36216bdf82dd9e03512c63cbac5c/recognition/arcface_torch/eval/verification.py#L179